In [1]:
from heol_humanoid import HeolHumanoid

In [304]:
heol = HeolHumanoid(simulator='vrep')

<DxlMotor name=l_hip_motor_z id=42 pos=0>
<DxlMotor name=l_thigh_x id=43 pos=0>
<DxlMotor name=l_ankle_x id=45 pos=0>
<DxlMotor name=r_hip_y id=51 pos=0>
<DxlMotor name=r_hip_motor_z id=52 pos=0>
<DxlMotor name=r_thigh_x id=53 pos=0>
<DxlMotor name=r_ankle_x id=55 pos=0>
<DxlMotor name=spine_z id=11 pos=0>
<DxlMotor name=chest_x id=12 pos=0>
<DxlMotor name=l_shoulder_x id=31 pos=0>
<DxlMotor name=l_upper_arm_z id=33 pos=0>
<DxlMotor name=r_shoulder_x id=21 pos=0>
<DxlMotor name=r_upper_arm_z id=23 pos=0>
<DxlMotor name=head_z id=13 pos=0>
<DxlMotor name=r_shoulder_motor_y id=22 pos=0>
<DxlMotor name=l_shoulder_motor_y id=32 pos=0>


In [305]:
from pypot.dynamixel import conversion
conversion.torque_max['XL-320']=3.0

In [306]:
conversion.torque_max

{'AX-12': 1.2,
 'AX-18': 1.8,
 'MX-106': 8.4,
 'MX-12': 1.2,
 'MX-28': 2.5,
 'MX-64': 6.0,
 'RX-28': 2.5,
 'RX-64': 4.0,
 'XL-320': 3.0}

###Marche équilibrée

In [147]:
heol.l_knee_x.goto_position(10,1)
heol.l_thigh_x.goto_position(-10,1)
heol.l_ankle_x.goto_position(10,1)
heol.l_foot_y.goto_position(-15,1)
heol.r_foot_y.goto_position(-10,1)
heol.l_hip_y.goto_position(15,1)
heol.r_hip_y.goto_position(-10,1,wait=True)
heol.r_thigh_x.goto_position(-40,1)
heol.r_knee_x.goto_position(20,1)
heol.r_ankle_x.goto_position(-10,1)
heol.r_hip_y.goto_position(0,1)
heol.r_foot_y.goto_position(0,1,wait=True)
heol.l_thigh_x.goto_position(30,2)
heol.l_ankle_x.goto_position(30,2)
heol.r_thigh_x.goto_position(-40,1)
heol.r_knee_x.goto_position(20,1)
heol.r_ankle_x.goto_position(-20,1,wait=True)
heol.r_hip_y.goto_position(15,1)
heol.r_foot_y.goto_position(15,1)
heol.chest_x.goto_position(-40,1)
heol.r_shoulder_x.goto_position(40,1)
heol.l_shoulder_x.goto_position(40,1,wait=True)
heol.l_ankle_x.goto_position(50,2)
heol.l_knee_x.goto_position(30,1,wait=True)
heol.l_thigh_x.goto_position(0,1)
heol.l_knee_x.goto_position(70,1)
heol.l_ankle_x.goto_position(70,2)
heol.l_hip_y.goto_position(5,2,wait=True)
heol.l_thigh_x.goto_position(-50,1)
heol.l_knee_x.goto_position(90,1)
heol.l_ankle_x.goto_position(40,2,wait=True)
heol.r_thigh_x.goto_position(-0,1)
heol.r_knee_x.goto_position(0,1)
heol.r_ankle_x.goto_position(-0,1)
heol.chest_x.goto_position(0,1)
heol.r_shoulder_x.goto_position(0,1)
heol.l_shoulder_x.goto_position(0,1,wait=True)
heol.l_thigh_x.goto_position(-40,1)
heol.l_knee_x.goto_position(0,2)
heol.l_ankle_x.goto_position(-40,2,wait=True)
heol.l_foot_y.goto_position(-5,1)
heol.r_thigh_x.goto_position(22,1)
heol.r_knee_x.goto_position(0,1)
heol.r_ankle_x.goto_position(22,1,wait=True)
heol.l_thigh_x.goto_position(-30,1)
heol.l_knee_x.goto_position(0,1)
heol.l_ankle_x.goto_position(-30,1,wait=True)
heol.r_thigh_x.goto_position(30,1)
heol.r_knee_x.goto_position(0,1)
heol.r_ankle_x.goto_position(30,1)


In [137]:
heol.l_ankle_x.goto_position(-30,1,wait=True)

In [254]:
for m in heol.motors:
    m.goto_position(0,1)

###Marche en déséquilibre

In [311]:
heol.r_shoulder_x.goto_position(30,0.5)
heol.l_shoulder_x.goto_position(30,0.5)

In [312]:
heol.l_ankle_x.goto_position(-20,0.1)
heol.l_knee_x.goto_position(40,0.1)
heol.l_thigh_x.goto_position(-40,0.1)
heol.r_ankle_x.goto_position(20,0.1)
heol.r_thigh_x.goto_position(20,0.1,wait=True)

heol.l_knee_x.goto_position(0,0.2)
heol.l_thigh_x.goto_position(-20,0.1)
heol.l_ankle_x.goto_position(-20,0.1)


In [315]:


heol.l_ankle_x.goto_position(0,0.1)
heol.l_thigh_x.goto_position(0,0.1)

heol.r_ankle_x.goto_position(-20,0.1)
heol.r_knee_x.goto_position(40,0.1)
heol.r_thigh_x.goto_position(-40,0.1,wait=True)


heol.l_thigh_x.goto_position(20,0.1)
heol.l_ankle_x.goto_position(20,0.1)

heol.r_knee_x.goto_position(0,0.1)
heol.r_thigh_x.goto_position(-20,0.1)
heol.r_ankle_x.goto_position(-20,0.1,wait=True)


In [314]:
heol.r_ankle_x.goto_position(0,0.1)
heol.r_thigh_x.goto_position(0,0.1)

heol.l_ankle_x.goto_position(-20,0.1)
heol.l_knee_x.goto_position(40,0.1)
heol.l_thigh_x.goto_position(-40,0.1,wait=True)


heol.r_thigh_x.goto_position(20,0.1)
heol.r_ankle_x.goto_position(20,0.1)

heol.l_knee_x.goto_position(0,0.1)
heol.l_thigh_x.goto_position(-20,0.1)
heol.l_ankle_x.goto_position(-20,0.1,wait=True)


In [298]:
for i in range(3):
    heol.r_ankle_x.goto_position(0,0.1)
    heol.r_thigh_x.goto_position(0,0.1)

    heol.l_ankle_x.goto_position(-20,0.1)
    heol.l_knee_x.goto_position(40,0.1)
    heol.l_thigh_x.goto_position(-40,0.1,wait=True)


    heol.r_thigh_x.goto_position(20,0.1)
    heol.r_ankle_x.goto_position(20,0.1)

    heol.l_knee_x.goto_position(0,0.1)
    heol.l_thigh_x.goto_position(-20,0.1)
    heol.l_ankle_x.goto_position(-20,0.1,wait=True)

    time.sleep(0.1)
    
    heol.l_ankle_x.goto_position(0,0.1)
    heol.l_thigh_x.goto_position(0,0.1)

    heol.r_ankle_x.goto_position(-20,0.1)
    heol.r_knee_x.goto_position(40,0.1)
    heol.r_thigh_x.goto_position(-40,0.1,wait=True)


    heol.l_thigh_x.goto_position(20,0.1)
    heol.l_ankle_x.goto_position(20,0.1)

    heol.r_knee_x.goto_position(0,0.1)
    heol.r_thigh_x.goto_position(-20,0.1)
    heol.r_ankle_x.goto_position(-20,0.1,wait=True)

In [223]:
heol.l_ankle_x.goto_position(20,0.2)
heol.l_thigh_x.goto_position(20,0.2)

In [227]:

heol.r_knee_x.goto_position(10,0.1)



In [229]:
heol.l_hip_y.goto_position(10,0.1)
heol.l_foot_y.goto_position(-10,0.1)


In [245]:
heol.l_thigh_x.goto_position(-20,0.1)
heol.l_ankle_x.goto_position(-20,0.1)

heol.r_thigh_x.goto_position(20,0.1)
heol.r_ankle_x.goto_position(20,0.1)


In [246]:
heol.l_thigh_x.goto_position(0,0.1)
heol.l_ankle_x.goto_position(0,0.1)



In [ ]:
heol.r_thigh_x.goto_position(-20,0.1)
heol.r_ankle_x.goto_position(-20,0.1)

In [316]:

heol.reset_simulation()

In [186]:
heol.r_ankle_x.goto_position(20,1)
heol.r_thigh_x.goto_position(20,1,wait=True)

In [150]:
heol.r_ankle_x.compliant = True
heol.r_foot_y.compliant = True

In [154]:
heol.r_knee_x.compliant = True
heol.l_knee_x.compliant = True

In [90]:
from pypot.primitive import Primitive
import time
class wait_primitive(Primitive):
    def run(self):
        for m in self.robot.motors:
            ecart = m.goal_position - m.present_position
            print ecart
        

In [91]:
wait = wait_primitive(heol)

In [92]:
wait.start()
time.sleep(1)

-1.05727784988e-06
0.00115288561428
0.458600986071
-0.0495459842183
-3.16034528435e-06
6.17553514193e-06
9.92917163123e-07
0.00758281731961
0.0115787971989
-0.00539134277615
-0.00135769196733
0.0115787971989
-0.0173681957983
0.0173681957983
-0.00130927699849
-0.0115787971989
-6.278755286e-06
4.80744656769e-09
0.0374334820713
0.0115787971989
-0.010652493423
0.00336482324212
-0.00157531586821


In [211]:

heol.reset_simulation()

In [303]:
heol.close()